In [5]:
from spotify import Client, OAuth
import pandas as pd

In [7]:
auth = OAuth('f716b10140c04e9997c89e1d094745f6', 'b9aa1c134fe244ebaed7b3cba541f6a7')
auth.request_client_credentials()

In [16]:
spotifyClient = Client(auth)
spotifyAPI = spotifyClient.api

In [8]:
def getTrackAttributes(trackIdList = []):
    print(trackIdList)
    trackJson = spotifyAPI.tracks_audio_features(trackIdList)
    trackList = trackJson['audio_features']
    trackList = filter(None, trackList)
    trackList = [x for x in trackList if x is not None]
    return trackList

In [9]:
def  getAttributeDF(trackFeatList):
    trackDF = pd.DataFrame(trackFeatList)
    trackDF = trackDF.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1)
    return trackDF

In [10]:
def getPlaylistTrack(play_id,user_id):
    tracks = spotifyAPI.user_playlist_tracks(playlist_id= play_id, user_id= user_id,fields='items(track.id,track.album.name,track.name,track.artists)')
    #tracks = list(filter(None, tracks))
    return tracks

In [11]:
def trackJSONtoDF(tracks,genre):
    flatData = []

    for i in range(len(tracks['items'])):
        trackObj = {}
        
        trackObj['id'] = tracks['items'][i]['track']['id']
        trackObj['album'] = tracks['items'][i]['track']['album']['name']
        trackObj['track_name'] = tracks['items'][i]['track']['name']
        trackObj['artist'] = tracks['items'][i]['track']['artists'][0]['name']
        trackObj['genre'] = genre
        if trackObj['id'] != None:
            flatData.append(trackObj)
    return pd.DataFrame(flatData)

In [12]:
def pipeLineFunc(play_id,user_id,genre):
    # Fetch playlist tracks
    plTracks = getPlaylistTrack(play_id,user_id)
    
    
    # Convert to DF
    plTracksDF =  trackJSONtoDF(plTracks,genre)

    # Get list of track ids
    trackIds = list(plTracksDF['id'])

    # Get tracks features
    trackFeat = getTrackAttributes(trackIds)

    # Convert JSON to DF
    trackFeatDF = getAttributeDF(trackFeat)

    # Merging 2 data frames
    resultDF = pd.concat([plTracksDF, trackFeatDF], axis=1, join='inner')

    # Droping duplicate ID columns
    resultDF = resultDF.T.drop_duplicates().T

    return resultDF

In [13]:
playListData = [{'user':'spotify','playlist_id':'37i9dQZF1DWWwzidNQX6jx','genre':'70sRock'},
{'user':'rb89vh0sf6cjkviw4mf1ymvww','playlist_id':'5VeYoZzAUNyLw4LaMtTz1v','genre':'2000sRock'}]


In [14]:
playlistDF = pd.DataFrame(playListData)
print(playlistDF)

       genre             playlist_id                       user
0    70sRock  37i9dQZF1DWWwzidNQX6jx                    spotify
1  2000sRock  5VeYoZzAUNyLw4LaMtTz1v  rb89vh0sf6cjkviw4mf1ymvww


In [17]:
trackDFMain = pd.DataFrame()
for i in range(len(playlistDF)):
    resDF = pipeLineFunc(playlistDF["playlist_id"][i],playlistDF["user"][i],playlistDF["genre"][i])
    trackDFMain = trackDFMain.append(resDF)

    

['2GAIycsMaDVtMtdvxzR2xI', '5CQ30WqJwcep0pYcV4AMNc', '40riOy7x9W7GXjyGp4pjAv', '5jzma6gCzYtKB1DbEwFZKH', '14XWXWv5FoCbFzLksawpEe', '6KTv0Z8BmVqM7DPxbGzpVC', '02DurCgOvDdX0uKEjqcl3W', '0LrwgdLsFaWh9VXIjBRe8t', '3Bh6uInhcVBVvLraGZdkKD', '1IqFh00G2kvvMm8pRMpehA', '52dm9op3rbfAkc1LGXgipW', '7pKfPomDEeI4TPT6EOYjn9', '7iN1s7xHE4ifF5povM6A48', '5MxNLUsfh7uzROypsoO5qe', '0SYRVn2YF7HBscQEmlkpTI', '2zYzyRzz6pRmhPzyfMEC8s', '6FsQrRpBLgsrFeAeiQqytm', '5SAUIWdZ04OxYfJFDchC7S', '4YMQXzscifAREG0a7KNGhB', '5xD021bM8VbqrhCVhxRozx', '5EWPGh7jbTNO2wakv8LjUI', '3ykSdTGmYPFl8pDBXer1zG', '0UAJH0k4k3slcE83a9UGCe', '3qT4bUD1MaWpGrTwcvguhb', '4KcH1ZRV2W1q7Flq0QqC76', '1JkZg3eMQTmTn93E8Yd3UL', '6PmckFG2BhEELc5OM7O7DS', '7kNNylJ1kswWbHeRM6UDuE', '7MRyJPksH3G2cXHN8UKYzP', '05uGBKRCuePsf43Hfm0JwX', '5HNCy40Ni5BZJFw1TKzRsC', '6hTcuIQa0sxrrByu9wTD7s', '2kkvB3RNRzwjFdGhaUA0tz', '4alHo6RGd0D3OUbTPExTHN', '3LoDeIbiR12sAznmpNEmKA', '2QgWuCtBpNIpl5trmKCxRf', '5UuikgHTxSRFRnC0zXx10i', '0Rp4akmfqBTpbDM6EoqtNy', '3IOQZRcEkp

In [18]:
uniquetrackDF = trackDFMain.drop_duplicates('id')

In [19]:
uniquetrackDF.to_csv('RockMusicInfoNew.csv')